# General E(2)-Equivariant Steerable CNNs  -  A concrete example


In [1]:
import os
import torch
import cv2
from e2cnn import gspaces
from e2cnn import nn

Finally, we build a **Steerable CNN** and try it MNIST.

Let's also use a group a bit larger: we now build a model equivariant to $8$ rotations.
We indicate the group of $N$ discrete rotations as $C_N$, i.e. the **cyclic group** of order $N$.
In this case, we will use $C_8$.

Because the inputs are still gray-scale images, the input type of the model is again a *scalar field*.

However, internally we use *regular fields*: this is equivalent to a *group-equivariant convolutional neural network*.

Finally, we build *invariant* features for the final classification task by pooling over the group using *Group Pooling*.

The final classification is performed by a two fully connected layers.

# The model

Here is the definition of our model:

In [2]:
class C8SteerableCNN(torch.nn.Module):
    
    def __init__(self, n_classes=10):
        
        super(C8SteerableCNN, self).__init__()
        
        # the model is equivariant under rotations by 45 degrees, modelled by C8
        self.r2_act = gspaces.Rot2dOnR2(N=8)
        #print("self.r2_act", self.r2_act)
        
        # the input image is a scalar field, corresponding to the trivial representation
        in_type = nn.FieldType(self.r2_act, 3*[self.r2_act.trivial_repr])
        #print("in_type", in_type)
        
        # we store the input type for wrapping the images into a geometric tensor during the forward pass
        self.input_type = in_type
        
        # convolution 1
        # first specify the output type of the convolutional layer
        # we choose 24 feature fields, each transforming under the regular representation of C8
        out_type = nn.FieldType(self.r2_act, 24*[self.r2_act.regular_repr])
        #print("out_type", out_type)
        self.block1 = nn.SequentialModule(
            nn.MaskModule(in_type, 128, margin=1),
            nn.R2Conv(in_type, out_type, kernel_size=7, padding=1, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block1', self.block1.out_type.size)
        
        # convolution 2
        # the old output type is the input type to the next layer
        in_type = self.block1.out_type
        # the output type of the second convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 48*[self.r2_act.regular_repr])
        self.block2 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block2', self.block2.out_type.size)
        self.pool1 = nn.SequentialModule(
            nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        )
        #print('pool1', self.pool1.out_type.size)
        
        # convolution 3
        # the old output type is the input type to the next layer
        in_type = self.block2.out_type
        # the output type of the third convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 48*[self.r2_act.regular_repr])
        self.block3 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block3', self.block3.out_type.size)
        
        # convolution 4
        # the old output type is the input type to the next layer
        in_type = self.block3.out_type
        # the output type of the fourth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 96*[self.r2_act.regular_repr])
        self.block4 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block4', self.block4.out_type.size)
        self.pool2 = nn.SequentialModule(
            nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        )
        #print('pool2', self.pool2.out_type.size)
        
        # convolution 5
        # the old output type is the input type to the next layer
        in_type = self.block4.out_type
        # the output type of the fifth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 96*[self.r2_act.regular_repr])
        self.block5 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block5', self.block5.out_type.size)
        
        # convolution 6
        # the old output type is the input type to the next layer
        in_type = self.block5.out_type
        # the output type of the sixth convolution layer are 64 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 64*[self.r2_act.regular_repr])
        self.block6 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=1, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block6', self.block6.out_type.size)
        self.pool3 = nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=1, padding=0)
        #print('pool3', self.pool3.out_type.size)
        
        self.gpool = nn.GroupPooling(out_type) # pool3.out_type
        
        # number of output channels
        c = self.gpool.out_type.size
        #print('gpool', c)
        
        # Fully Connected
        self.fully_net = torch.nn.Sequential(
            torch.nn.Linear(c*25*25, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ELU(inplace=True),
            torch.nn.Linear(64, n_classes),
        )
    
    def forward(self, input: torch.Tensor):
        # wrap the input tensor in a GeometricTensor
        # (associate it with the input type)
        #print("input.shape", input.shape)
        x = nn.GeometricTensor(input, self.input_type)
        
        # apply each equivariant block
        
        # Each layer has an input and an output type
        # A layer takes a GeometricTensor in input.
        # This tensor needs to be associated with the same representation of the layer's input type
        #
        # The Layer outputs a new GeometricTensor, associated with the layer's output type.
        # As a result, consecutive layers need to have matching input/output types
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool1(x)
        
        x = self.block3(x)
        x = self.block4(x)
        x = self.pool2(x)
        
        x = self.block5(x)
        x = self.block6(x)
        
        # pool over the spatial dimensions
        x = self.pool3(x)
        #print("x.shape1", x.shape)
        
        # pool over the group
        x = self.gpool(x)
        #print("x.shape2", x.shape)

        # unwrap the output GeometricTensor
        # (take the Pytorch tensor and discard the associated representation)
        x = x.tensor
        #print("x.shape3", x.shape)
        
        # classify with the final fully connected layers)
        x = self.fully_net(x.reshape(x.shape[0], -1)) # x.shape[1]?
        
        return x

Let's try the model on *rotated* MNIST

In [3]:
# # download the dataset
# !wget -nc http://www.iro.umontreal.ca/~lisa/icml2007data/mnist_rotation_new.zip
# # uncompress the zip file
# !unzip -n mnist_rotation_new.zip -d mnist_rotation_new

In [4]:
# import requests
# url = 'http://www.iro.umontreal.ca/~lisa/icml2007data/mnist_rotation_new.zip'
# doc = requests.get(url)
# with open('mnistrot.zip', 'wb') as f:
#     f.write(doc.content)

In [3]:
from torch.utils.data import Dataset
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose

import numpy as np

from PIL import Image

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Build the dataset

In [4]:
class DOTARotDataset(Dataset):
    
    def __init__(self, mode, transform=None):
        assert mode in ['train', 'test']
        
        basedir = 'C:/Users/Admin/Desktop/data/DOTAv1.0/'
        
        if mode == "train":
            file = basedir+"chips_train/"
        else:
            file = basedir+"chips_val/"
        
        self.transform = transform
        self.images = []
        self.labels = []
        new_image_width = 128
        new_image_height = 128
        color = (0,0,0)
        self.classdict = {}
        self.classval = 0

        #data = np.loadtxt(file, delimiter=' ')
        data = []
        
        for root, dirs, filenames in os.walk(file, topdown=False):
            pass
        for d in dirs:
            for root, dirs, filenames in os.walk(file+d, topdown=False):
                pass
            print(d, len(filenames))
            for f in filenames:
                imgstr = file+d+"/"+f
                #img = Image.open(imgstr)
                #arr = np.asarray(img)
                img = cv2.imread(imgstr)
                old_image_height, old_image_width, channels = img.shape
                #print('old', old_image_height, old_image_width)
                if img.shape[0] < img.shape[1]:
                    result1 = np.full((img.shape[1],img.shape[1], channels), color, dtype=np.uint8)
                    #print('shape', result1.shape)
                    # compute center offset
                    #x_center = np.abs(new_image_width - old_image_width) // 2
                    y_center = np.abs(img.shape[1] - old_image_height) // 2
                    #print('y_center', y_center)
                    # copy img image into center of result image
                    result1[y_center:y_center+old_image_height, :] = img
                elif img.shape[0] > img.shape[1]:
                    result1 = np.full((img.shape[0],img.shape[0], channels), color, dtype=np.uint8)
                    #print('shape', result1.shape)
                    # compute center offset
                    x_center = np.abs(img.shape[0] - old_image_width) // 2
                    #y_center = np.abs(new_image_height - old_image_height) // 2
                    #print('x_center', x_center)
                    # copy img image into center of result image
                    result1[:, x_center:x_center+old_image_width] = img
                else:
                    result1 = img[:,:,:]
                # check if we need to resize
                if not (result1.shape[0] == 128 and result1.shape[1] == 128):
                    #print('result1.shape1', result1.shape)
                    result1 = cv2.resize(result1, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
                    #print('result1.shape2', result1.shape)
                res2 = result1.reshape(-1, 128, 128).astype(np.float32) # = arr[:, :-1].reshape(-1, 128, 128)
                #print('res2.shape', res2.shape)
                
                # ToTensor screws up the order, so we have to undo it:
                # https://discuss.pytorch.org/t/torchvision-totensor-dont-change-channel-order/82038/2
                #res2 = res2.permute((1, 2, 0)).contiguous()
                
                self.images.append(res2)
        
                # labels
                if self.classdict.get(d, False):
                    self.labels.append(self.classdict[d])
                else: # it's not in self.classdict yet
                    self.classdict[d] = self.classval
                    self.classval += 1
        self.num_samples = len(self.labels)
    
    def __getitem__(self, index):
        image, label = self.images[index], self.labels[index]
        #image = Image.fromarray(image)
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.labels)

# images are padded to have shape 29x29.
# this allows to use odd-size filters with stride 2 when downsampling a feature map in the model
pad = Pad((0, 0, 1, 1), fill=0)

# to reduce interpolation artifacts (e.g. when testing the model on rotated images),
# we upsample an image by a factor of 3, rotate it and finally downsample it again
resize1 = Resize(128*3)
resize2 = Resize(128)

totensor = ToTensor()

Let's build the model

In [5]:
model = C8SteerableCNN(n_classes=15).to(device)

C:\Users\Admin\Desktop\school\e2cnn_experiments\e2cnn\nn\modules\r2_conv\basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen/native/IndexingUtils.h:30.)
  full_mask[mask] = norms.to(torch.uint8)


In [179]:
from torchsummary import summary
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
        MaskModule-1          [-1, 3, 128, 128]               0
SingleBlockBasisExpansion-2             [-1, 8, 1, 49]               0
BlocksBasisExpansion-3                [-1, 3, 49]               0
            R2Conv-4        [-1, 192, 124, 124]               0
       BatchNorm3d-5      [-1, 24, 8, 124, 124]              48
    InnerBatchNorm-6        [-1, 192, 124, 124]               0
              ReLU-7        [-1, 192, 124, 124]               0
  SequentialModule-8        [-1, 192, 124, 124]               0
SingleBlockBasisExpansion-9             [-1, 8, 8, 25]               0
SingleBlockBasisExpansion-10             [-1, 8, 8, 25]               0
SingleBlockBasisExpansion-11             [-1, 8, 8, 25]               0
SingleBlockBasisExpansion-12             [-1, 8, 8, 25]               0
SingleBlockBasisExpansion-13             [-1, 8, 8, 25]        

The model is now randomly initialized. 
Therefore, we do not expect it to produce the right class probabilities.

However, the model should still produce the same output for rotated versions of the same image.
This is true for rotations by multiples of $\frac{\pi}{2}$, but is only approximate for rotations by $\frac{\pi}{4}$.

Let's test it on a random test image:
we feed eight rotated versions of the first image in the test set and print the output logits of the model for each of them.

In [98]:
# build the test set
raw_mnist_test = DOTARotDataset(mode='test')

baseball-diamond 73
basketball-court 111
bridge 2
ground-track-field 19
harbor 2307
helicopter 18
large-vehicle 3164
plane 1543
roundabout 50
ship 15846
small-vehicle 7673
soccer-ball-field 52
storage-tank 58
swimming-pool 1019
tennis-court 1133


In [143]:
# retrieve the first image from the test set
x, y = next(iter(raw_mnist_test))

print(x.shape)
print(raw_mnist_test[3530])

(3, 128, 128)
(array([[[  0.,   0.,   0., ...,  57.,  12.,  13.],
        [ 11.,   3.,   4., ...,  38.,  29.,  17.],
        [ 12.,   4.,   9., ...,   0.,   0.,   0.],
        ...,
        [  5.,   4.,   7., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,  12.,  46.,  35.],
        [ 28.,  29.,  15., ...,  50.,  49.,   9.]],

       [[  8.,   7.,  38., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   6.,  37.,  23.],
        [ 17.,  24.,   7., ...,  14.,  14.,  22.],
        ...,
        [ 58.,  63.,  58., ..., 141., 136., 149.],
        [148., 144., 157., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,  49.,  50.,  44.]],

       [[ 47.,  63.,  57., ..., 143., 138., 153.],
        [151., 147., 156., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,  29.,  40.,  32.],
        ...,
        [  0.,   0.,   0., ...,  39.,  29.,  20.],
        [ 11.,  50.,  45., ..., 134., 129., 130.],
        [136., 131., 141., ...,   0.,   0.,   0.]]], dtype=float32), 7)


In [184]:
def test_model(model: torch.nn.Module, x: Image):
    # evaluate the `model` on 8 rotated versions of the input image `x`
    model.eval()
    
    wrmup = model(torch.randn(1, 3, 128, 128).to(device))
    del wrmup
    
    #x = resize1(pad(x))
    
    print()
    print('##########################################################################################')
    header = 'angle |  ' + '  '.join(["{:6d}".format(d) for d in range(10)])
    print(header)
    with torch.no_grad():
        for r in range(8):
            #print(np.min(x), np.max(x), x.shape)
            intimg = x.astype(np.uint8)
            #print(type(intimg))
            print(np.min(intimg), np.max(intimg), intimg.shape)
            img = Image.fromarray(intimg)
            #print(np.min(img), np.max(img))
            rotimg = img.rotate(r*45., Image.BILINEAR)
            #print(rotimg.shape)
            x_transformed = totensor(rotimg).reshape(-1, 3, 128, 128)
            x_transformed = x_transformed.to(device)

            y = model(x_transformed)
            y = y.to('cpu').numpy().squeeze()
            
            angle = r * 45
            print("{:5d} : {}".format(angle, y))
    print('##########################################################################################')
    print()

In [185]:
# evaluate the model
test_model(model, x)


##########################################################################################
angle |       0       1       2       3       4       5       6       7       8       9
0 249 (3, 128, 128)


TypeError: Cannot handle this data type: (1, 1, 128), |u1

The output of the model is already almost invariant.
However, we still observe small fluctuations in the outputs.

This is because the model contains some operations which might break equivariance.
For instance, every convolution includes a padding of $2$ pixels per side. This is adds information about the actual orientation of the grid where the image/feature map is sampled because the padding is not rotated with the image. 

During training, the model will observe rotated patterns and will learn to ignore the noise coming from the padding.

So, let's train the model now.
The model is exactly the same used to train a normal *PyTorch* architecture:

In [6]:
# Prep the training dataset
train_transform = Compose([
    #pad,
    #resize1,
    #RandomRotation(180, resample=Image.BILINEAR, expand=False),
    #resize2,
    #totensor
])

mnist_train = DOTARotDataset(mode='train', transform=train_transform)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=16)

baseball-diamond 58
basketball-court 50
bridge 2
ground-track-field 18
harbor 703
helicopter 18
large-vehicle 2624
plane 1428
roundabout 44
ship 652
small-vehicle 6189
soccer-ball-field 48
storage-tank 55
swimming-pool 967
tennis-court 682


In [7]:
# Prep the testing dataset
test_transform = Compose([
    #pad,
    #totensor,
])
mnist_test = DOTARotDataset(mode='test', transform=test_transform)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=16)

baseball-diamond 74
basketball-court 99
bridge 297
ground-track-field 38
harbor 1977
helicopter 66
large-vehicle 1254
plane 1722
roundabout 45
ship 8141
small-vehicle 3270
soccer-ball-field 43
storage-tank 443
swimming-pool 433
tennis-court 382


In [8]:
# Initialize the model
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-5)

In [13]:
# get initial performance with random weights
total = 0
correct = 0
with torch.no_grad():
    model.eval()
    for i, (x, t) in enumerate(test_loader):
        if i%100==0:
            print(i)

        x = x.to(device)
        t = t.to(device)

        y = model(x)
        
        _, prediction = torch.max(y.data, 1)
        total += t.shape[0]
        correct += (prediction == t).sum().item()
print(f"epoch {epoch} | test accuracy: {correct/total*100.}")

epoch 0 | test accuracy: 6.86993650098533


In [ ]:
for epoch in range(10):
    model.train()
    for i, (x, t) in enumerate(train_loader):

        optimizer.zero_grad()
        #print(x.shape)

        x = x.to(device)
        t = t.to(device)

        y = model(x)

        loss = loss_function(y, t)

        loss.backward()

        optimizer.step()
    
    #if epoch % 10 == 0:
    total = 0
    correct = 0
    with torch.no_grad():
        model.eval()
        for i, (x, t) in enumerate(test_loader):

            x = x.to(device)
            t = t.to(device)

            y = model(x)

            _, prediction = torch.max(y.data, 1)
            total += t.shape[0]
            correct += (prediction == t).sum().item()
    print(f"epoch {epoch} | train accuracy: {correct/total*100.}")
    print(f"epoch {epoch} | train loss: {correct/total*100.}")
    print(f"epoch {epoch} | test accuracy: {correct/total*100.}")
    print(f"epoch {epoch} | test loss: {correct/total*100.}")


In [34]:
# retrieve the first image from the test set
x, y = next(iter(raw_mnist_test))


# evaluate the model
test_model(model, x)


##########################################################################################
angle |       0       1       2       3       4       5       6       7       8       9
    0 : [-0.2373 -0.5674 -1.0786 -0.5599  0.1627  0.3779  4.1019  0.4574 -1.1484 -0.3254]
   45 : [-0.5487 -0.4267 -1.1523 -0.7452  0.1651  0.327   4.0815  0.5293 -0.8953 -0.3933]
   90 : [-0.2373 -0.5674 -1.0786 -0.5599  0.1627  0.3779  4.1019  0.4574 -1.1484 -0.3254]
  135 : [-0.5487 -0.4267 -1.1523 -0.7452  0.1651  0.327   4.0815  0.5293 -0.8953 -0.3933]
  180 : [-0.2373 -0.5674 -1.0786 -0.5599  0.1627  0.3779  4.1019  0.4574 -1.1484 -0.3254]
  225 : [-0.5487 -0.4267 -1.1523 -0.7452  0.1651  0.3271  4.0815  0.5293 -0.8953 -0.3933]
  270 : [-0.2373 -0.5674 -1.0786 -0.5599  0.1627  0.3779  4.1019  0.4574 -1.1484 -0.3254]
  315 : [-0.5487 -0.4267 -1.1523 -0.7452  0.1651  0.327   4.0815  0.5293 -0.8953 -0.3933]
##########################################################################################

